# load packages

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from matplotlib import pyplot as plt

# get the data

get csv's

In [ ]:
data = pd.read_csv("../input/g2net-gravitational-wave-detection/training_labels.csv")
sample = pd.read_csv("../input/g2net-gravitational-wave-detection/sample_submission.csv")

plot the npy file

In [ ]:
plot = np.load("../input/g2net-gravitational-wave-detection/train/0/0/0/00000e74ad.npy")
plt.plot(plot[0])

define generator

In [ ]:
def generator():
    for i in range(1000):#range(len(data["id"])): I only have 1000 to make training faster
        file = "../input/g2net-gravitational-wave-detection/train/"
        file += data["id"][i][0] + "/" + data["id"][i][1] + "/" + data["id"][i][2] + "/"
        file += data["id"][i] + ".npy"
        
        x_temp = np.load(file)
        x = []
        
        for w in range(len(x_temp[0])):
            x.append((x_temp[0][w]+x_temp[1][w]+x_temp[2][w])/3) # get the mean to smooth it
                
        
        y = data["target"][i]
        
        yield np.array(x), np.array(y)

it works!!!

In [ ]:
i = generator()
plt.plot(next(i)[0]) # same image
print(np.shape(next(i)))
print(np.shape(next(i)[0]))

add a better model below

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=(4096,)),
        tf.keras.layers.Dense(64),
        tf.keras.layers.Dense(1, "sigmoid")
    ]
)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.BinaryCrossentropy())
model.summary()

In [ ]:
gen = generator()
batch_size=10

xs = []
ys = []

for x, y in gen:
    model.train_on_batch(np.reshape(x,(1,4096)),np.reshape(y,(1,1)))

# predictions

In [ ]:
for i in range(len(sample["id"])):
    file = "../input/g2net-gravitational-wave-detection/test/"
    file += sample["id"][i][0] + "/" + sample["id"][i][1] + "/" + sample["id"][i][2] + "/"
    file += sample["id"][i] + ".npy"
        
    x_temp = np.load(file)
    x = []
        
    for w in range(len(x_temp[0])):
        x.append((x_temp[0][w]+x_temp[1][w]+x_temp[2][w])/3) # get the mean to smooth it
                
    sample["target"][i] = model(np.reshape(x,(1,4096)))

In [ ]:
sample.to_csv('submission.csv', index = False)
sample

I could not finish it fast but edit this and tell me how it goes